In [1]:
# Change directory to level up to import functions
import os
os.chdir('..')

import warnings

# Suppress all warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np

In [3]:
import yaml

file_path = 'conf/duplicated_player_names.yaml'

# Load the YAML file
with open(file_path, 'r') as file:
    duplicated_player_names = yaml.safe_load(file)

In [4]:
premier_league_goals = pd.read_csv('data/premier_league_goals/combined_seasons/premier_league_goals.csv')
premier_league_assists = pd.read_csv('data/premier_league_assists/combined_seasons/premier_league_assists.csv')
championship_goals = pd.read_csv('data/championship_goals/combined_seasons/championship_goals.csv')
championship_assists = pd.read_csv('data/championship_assists/combined_seasons/championship_assists.csv')


In [6]:
def rename_duplicated_players(df, duplicated_player_names, league):
    """
    Rename players in the DataFrame based on duplicated player names.
    
    Parameters:
    df (pd.DataFrame): The DataFrame containing player statistics (e.g., goals).
    duplicated_player_names (pd.DataFrame): A DataFrame containing duplicated player names with columns 'Player', 'Team', 'Season', and 'Rename'.
    
    Returns:
    pd.DataFrame: The modified DataFrame with updated player names.
    """
    for player_info in duplicated_player_names[league]:
        # Apply conditions to match 'Player', 'Team', and 'Season'
        condition = (
            (df['Player'] == player_info['Player']) &
            (df['Team'] == player_info['Team']) 
        )
        
        # Replace player name with 'Rename' if conditions match
        df.loc[condition, 'Player'] = player_info['Rename']
    
    return df

In [7]:
# Join PL Data

# Rename dupliacate player names
premier_league_goals= rename_duplicated_players(df=premier_league_goals, duplicated_player_names=duplicated_player_names, league='premier_league')
premier_league_assists= rename_duplicated_players(df=premier_league_assists, duplicated_player_names=duplicated_player_names, league='premier_league')

# Perform a full outer join
premier_league_merged = pd.merge(premier_league_assists, premier_league_goals,
                     on=['Player', 'Country', 'Team', 'Season', 'season_start'],
                     how='outer')

# Fill missing values with 0
premier_league_merged['Assists'] = premier_league_merged['Assists'].fillna(0)
premier_league_merged['Goals'] = premier_league_merged['Goals'].fillna(0)


# Select final columns
premier_league_merged = premier_league_merged[['Player', 'Country', 'Team', 'Assists', 'Season', 'season_start', 'Goals']]


# Check for NaN values and fill them with a placeholder (if needed)
premier_league_merged['Team'] = premier_league_merged['Team'].fillna('Unknown')  # Fill NaNs with a placeholder
premier_league_merged['Team'] = premier_league_merged['Team'].astype(str)  # Ensure all entries are strings

# Group by 'Player', 'Season', and 'season_start'
# This is for players who played for multiple clubs in the season
premier_league_merged = (premier_league_merged.groupby(['Player', 'Season', 'Country', 'season_start'])
          .agg({
              'Team': lambda x: ' / '.join(sorted(set(x))),
              'Assists': 'sum',
              'Goals': 'sum'
          })
          .reset_index())



In [8]:
# Join Championship Data
# Rename dupliacate player names
championship_goals= rename_duplicated_players(df=championship_goals, duplicated_player_names=duplicated_player_names, league='championship')
championship_assists= rename_duplicated_players(df=championship_assists, duplicated_player_names=duplicated_player_names, league='championship')


# Perform a full outer join
championship_merged = pd.merge(championship_goals, championship_assists,
                     on=['Player', 'Country', 'Team', 'Season', 'season_start'],
                     how='outer')

# Fill missing values with 0
championship_merged.loc[championship_merged['season_start'] >= 2014, 'Assists'] = championship_merged.loc[championship_merged['season_start'] > 2014, 'Assists'].fillna(0)
championship_merged['Goals'] = championship_merged['Goals'].fillna(0)


# Select final columns
championship_merged = championship_merged[['Player', 'Country', 'Team', 'Assists', 'Season', 'season_start', 'Goals']]

# Group by 'Player', 'Season', and 'season_start'
# This is for players who played for multiple clubs in the season
championship_merged = (championship_merged.groupby(['Player', 'Season', 'Country', 'season_start'])
          .agg({
              'Team': lambda x: ' / '.join(sorted(set(x))),
              'Assists': 'sum',
              'Goals': 'sum'
          })
          .reset_index())


In [9]:
# Merge premier league and championship
# Create a lagged season_start in the championship DataFrame
championship_merged['lagged_season_start'] = championship_merged['season_start'] + 1

# Perform inner join
pl_champ_merged = pd.merge(
    premier_league_merged,
    championship_merged,
    left_on=['Player', 'Country', 'season_start'],
    right_on=['Player', 'Country', 'lagged_season_start'],
    suffixes=('_premier_league', '_championship'),
    how='inner'
)

# Drop the lagged_season_start column if not needed
pl_champ_merged = pl_champ_merged.drop(columns=['lagged_season_start'])

In [10]:
# Check for dupes
# Step 3: Check for duplicates in the result DataFrame
duplicates_result = (pl_champ_merged
                     .groupby(['Player', 'Country', 'season_start_premier_league'])
                     .size()
                     .reset_index(name='count'))

# Filter for duplicates (count > 1)
duplicates_result = duplicates_result[duplicates_result['count'] > 1]


In [11]:
# Check if in same team in PL as Championship
# Create the same_team column
pl_champ_merged['same_team'] = np.where(
    pl_champ_merged['Team_premier_league'] == pl_champ_merged['Team_championship'], 
    1, 
    0
)

In [12]:
# Rename columns
pl_champ_merged.rename(columns={
    'Season_premier_league': 'Season (PL)',
    'season_start_premier_league': 'Season Start (PL)',
    'Season_championship': 'Season (Champ.)',
    'season_start_championship': 'Season Start (Champ.)',
    'Team_premier_league': 'Team (PL)',
    'Assists_premier_league': 'Assists (PL)',
    'Goals_premier_league': 'Goals (PL)',
    'Team_championship': 'Team (Champ.)',
    'Assists_championship': 'Assists (Champ.)',
    'Goals_championship': 'Goals (Champ.)',
    'same_team': 'With Promoted Team',
}, inplace=True)

In [13]:
# Test - check promoted teams correct
# Grouping by 'Team (Champ)' and 'Season (Champ)', counting occurrences
grouped_df = pl_champ_merged[pl_champ_merged['With Promoted Team']==1].groupby(['Season Start (Champ.)','Team (Champ.)']).size().reset_index(name='Count')

# Sorting by 'Team (Champ)' and 'Season (Champ)'
sorted_grouped_df = grouped_df.sort_values(by=['Season Start (Champ.)','Team (Champ.)'])
sorted_grouped_df.head(10)


,Season Start (Champ.),Team (Champ.),Count
0,2000,Blackburn Rovers,9
1,2000,Bolton Wanderers,9
2,2000,Fulham FC,5
3,2001,Birmingham City,5
4,2001,Manchester City,9
5,2001,West Bromwich Albion,9
6,2002,Leicester City,7
7,2002,Portsmouth FC,8
8,2002,Wolverhampton Wanderers,10
9,2003,Crystal Palace,6


In [16]:
pl_champ_merged.sort_values('Goals (PL)',ascending=False)

,Player,Season (PL),Country,Season Start (PL),Team (PL),Assists (PL),Goals (PL),Season (Champ.),Season Start (Champ.),Team (Champ.),Assists (Champ.),Goals (Champ.),With Promoted Team
58,Andrew Johnson,2004-2005,England,2004,Crystal Palace,3.0,21.0,2003-2004,2003,Crystal Palace,0.0,27.0,1
151,Charlie Austin,2014-2015,England,2014,Queens Park Rangers,7.0,18.0,2013-2014,2013,Queens Park Rangers,0.0,17.0,1
239,Darren Bent,2005-2006,England,2005,Charlton Athletic,4.0,18.0,2004-2005,2004,Ipswich Town,0.0,20.0,0
714,Patrick Bamford,2020-2021,England,2020,Leeds United,7.0,17.0,2019-2020,2019,Leeds United,2.0,16.0,1
917,Yakubu Ayegbeni,2011-2012,Nigeria,2011,Blackburn Rovers,2.0,17.0,2010-2011,2010,Leicester City,0.0,11.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
788,Sam Byram,2016-2017,England,2016,West Ham United,1.0,0.0,2015-2016,2015,Leeds United,1.0,3.0,0
182,Colin Hendry,2001-2002,Scotland,2001,Bolton Wanderers,1.0,0.0,2000-2001,2000,Bolton Wanderers,0.0,3.0,1
791,Sam Vokes,2011-2012,Wales,2011,Wolverhampton Wanderers,1.0,0.0,2010-2011,2010,Sheffield United,0.0,2.0,0
184,Collin Quaner,2017-2018,Germany,2017,Huddersfield Town,4.0,0.0,2016-2017,2016,Huddersfield Town,0.0,2.0,1
